In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
output_path = '/home/ubuntu/Documents/result_small'

In [21]:
import os
os.chdir('/home/FRACTAL/naquib.alam/fastai/old')
# This file contains all the main external libs we'll use
from fastai.imports import *

In [22]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from pathlib import *
import pandas as pd
import numpy as np

In [23]:
#checking wheteher cuda and cudnn are available. Cuda is necessary & Cudnn is optional, it makes computation faster
torch.cuda.is_available()

True

In [24]:
torch.backends.cudnn.enabled

True

In [25]:
#using  device 1-gpu
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [26]:
#checking the data
input_path = '/home/ubuntu/Documents/'
# classes = pd.read_csv(input_path+'scene_classes.csv',header=None)
# classes.rename(columns={0:'label_id',2:'class',1:'to_drop'},inplace=True)
# classes.drop('to_drop',inplace=True,axis=1)

labels = pd.read_csv(input_path+'label.csv')
# labels.drop('Unnamed: 0',inplace=True,axis=1)

# labels['label_id'] = labels['label_id'].astype(str)
# classes['label_id']= classes['label_id'].astype(str)

# labels = labels.merge(classes,how='left')
# labels = labels[~labels['class'].isna()]
# labels.reset_index(drop=True,inplace=True)

labels_model = labels[['image_id','class']]

FileNotFoundError: [Errno 2] File b'/home/ubuntu/Documents/label.csv' does not exist: b'/home/ubuntu/Documents/label.csv'

In [ ]:
labels_model['class'].unique()

In [ ]:
def get_data(f_model,sz,bs,val_idxs,aug_tfms,root_path,train_folder,trn_idxs,label_csv,test_folder,skip_header,augmentation_required):
    if (augmentation_required):
        tfms = tfms_from_model(f_model, sz,aug_tfms=aug_tfms,max_zoom=1.1)
    else:
        tfms =  tfms_from_model(f_model, sz)
    
    label_csv = label_csv.loc[np.concatenate((trn_indx,val_idxs)),:]#.drop((label_csv.index != val_idxs) | (label_csv.index != trn_idxs))
    
    label_csv.to_csv(str(root_path)+'temp_label_file.csv',index=False)
    
    return ImageClassifierData.from_csv_noise(root_path, train_folder,str(root_path)+'temp_label_file.csv',bs=bs,tfms=tfms,
                     val_idxs=val_idxs,test_name=test_folder,skip_header=skip_header)

In [ ]:
def exp_name(output_path,freeze,model_name,lrn,sz):
    return output_path+freeze+'_'+'_'+model_name+'_lr'+str(lrn)+'_sz'+str(sz)

In [ ]:
os.listdir("../../Documents/result_small/temp_model_store/")

In [ ]:
label_csv = labels_model.copy()
#a root path of the data (used for storing trained models, precomputed values, etc)
root_path=r'/home/ubuntu/Documents/result_small/temp_model_store/'
#a name of the folder in which training images are contained.
train_folder = r'/home/ubuntu/Documents/train/'
#a name of the folder which contains test images.
test_folder= r'/home/ubuntu/share/stage-3/stage3_image_data/test/test/'

In [ ]:
#for generating the validation indexes
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
labels_splits = labels.copy()
X= labels['class'].astype(str)
y=labels['class'].astype(str)

for train_index, test_index in sss.split(X, y):
    trn_indx = train_index
    val_idxs = test_index

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
labels_splits = labels.copy()
X = labels.loc[val_idxs, 'class'].astype(str)
y = labels.loc[val_idxs, 'class'].astype(str)

for train_index, test_index in sss.split(X, y):
    trn_indx = train_index
    val_idxs = test_index

In [ ]:
f_model=resnet50
sz=256
aug_tfms=transforms_side_on
max_zoom=1.1
bs=32
#use this if we dont need augmentation
augmentation_required = True

model_name=str(f_model).split(' ')[1]

In [ ]:
label_csv.loc[np.concatenate((trn_indx,val_idxs)),:].shape
# np.concatenate((trn_indx,val_idxs)).shape

In [ ]:
#making data object
data = get_data(f_model,sz,bs,val_idxs,aug_tfms,root_path,train_folder,trn_indx,label_csv,test_folder,skip_header=True,augmentation_required=augmentation_required)

In [ ]:
def weighted_f1(y_pred, y_true):
    y = [1 if i == "real" else 0 for i in y_true]
    f1R = fbeta(y_pred, torch.ByteTensor(np.array(y)).cuda(), thresh=0.5, beta=1)
    f1R *= np.sum(y)
    
    y = [0 if i == "real" else 1 for i in y_true]
    f1F = fbeta(1-y_pred, torch.ByteTensor(np.array(y)).cuda(), thresh=0.5, beta=1)
    f1F *= np.sum(y)
    
    return (f1F+f1R)/len(y)

In [ ]:
trn_label_count_f = np.sum(labels.loc[trn_indx, "class"].apply(lambda x: 1 if x == "fake" else 0))
trn_label_count_r = np.sum(labels.loc[trn_indx, "class"].apply(lambda x: 1 if x == "real" else 0))
trn_weights = [trn_label_count_r, trn_label_count_f]/(trn_label_count_f+trn_label_count_r)

In [ ]:
torch.cuda.set_device(0)
torch.cuda.current_device()
learn = ConvLearner.pretrained(f_model, data,precompute=False, metrics=[accuracy, f1])
learn.crit = nn.CrossEntropyLoss(weight=torch.FloatTensor(trn_weights).cuda())
# lrf=learn.lr_find()

# learn.sched.plot()


In [ ]:
learn.freeze()

In [ ]:
lrn=1e-3

In [ ]:
lrn

In [ ]:
learn.fit(lrn, 3, cycle_len=1, cycle_mult=2)

In [ ]:
preds, y = learn.TTA(8, is_test=False)
# learn.predict()
# preds, y = learn.predict_with_targs(is_test=False)

In [ ]:
# np.sum(preds[:,0,:],axis=0)
# learn.TTA()
# learn.data.val_ds.fnames


In [ ]:
pr = []
pk = np.mean(np.exp(preds), 0)
for p in pk:
    pr.append(p[1] > 0.8)
# for p in range(preds.shape[1]):
#     pr.append(np.mean(np.exp(preds[:,p,:],axis=0)[1]) > 0.5)
pr = np.array(pr)

In [ ]:
# pr.shape

In [ ]:
df = pd.DataFrame(pk, columns=["Real", "Fake"])
df["Filename"] = learn.data.val_ds.fnames
df.to_csv("val_noisepred.csv", index=False)

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr , tpr , thresh = roc_curve(y , pk[:,1])
roc_auc = auc(fpr , tpr)

plt.plot(fpr , tpr , 'b' , label = 'AUC = %0.2f' %roc_auc)
plt.show()

In [ ]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

In [ ]:
threshold = Find_Optimal_Cutoff(y, pk[:,1])
print(threshold)

In [ ]:
threshold

In [ ]:
from sklearn.metrics import f1_score
f1_score(pk[:,1] > 1-threshold[0], y, average="weighted")

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(pk[:,1] > 1-threshold[0], y)

In [ ]:
# data.val_ds.fnames
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresh[optimal_idx]
optimal_threshold

In [ ]:
from sklearn.metrics import f1_score
f1_score(pr.astype(int), y)

In [ ]:
plt.hist(pk[:,1],20)

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix() 
# interp.plot_top_losses(21, figsize=(15,11)) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, pr.astype(int))
plot_confusion_matrix(cm, data.classes)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask((preds == data.val_y)==is_correct & (data.val_y == y), mult)

In [ ]:
probs = pk[:,1]
PATH = ""
plot_val_with_title(most_by_correct(0, False), "Most incorrect fake")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect real")

In [ ]:
learn.save(exp_name(output_path,'frz',model_name,lrn,sz))

In [ ]:
#loading the previous saved model and unfreezing the network for more training
# learn.load(exp_name(output_path,'frz',model_name,lrn,sz))
learn.load("../../../result_smallfrz__resnet50_lr0.001_sz256")
# learn.unfreeze()
# learn.bn_freeze(True)

lr=np.array([lrn/100,lrn/10,lrn])

# learn.fit(lr, 3, cycle_len=1,cycle_mult=2)
# learn.save(exp_name(output_path,'unfrz',model_name,lrn,sz))


In [ ]:
learn.load("../../../result_smallunfrz__resnet50_lr0.001_sz256")

In [ ]:
sz=512

learn.set_data(get_data(f_model,sz,8,val_idxs,aug_tfms,root_path,train_folder,trn_indx,label_csv,test_folder,skip_header=True,augmentation_required=augmentation_required))
learn.freeze()
# learn.fit(lr, 3, cycle_len=1, cycle_mult=2)
learn.load("../../../result_smallfrz__resnet50_lr0.001_sz512")

In [ ]:
# learn.save(exp_name(output_path,'frz',model_name,lrn,sz))

In [ ]:
learn.unfreeze()
# learn.bn_freeze(True)
learn.fit(lr, 2, cycle_len=1, cycle_mult=2)

In [ ]:
# learn.save(exp_name(output_path,'unfrz',model_name,lrn,sz))

In [ ]:
learn.unfreeze()
learn.load(exp_name(output_path,'unfrz',model_name,lrn,sz))

In [ ]:
exp_name(output_path,'unfrz',model_name,lrn,sz)

In [ ]:
learn.fit(lr, 2, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save(exp_name(output_path,'unfrz',model_name,lrn,sz))

In [ ]:
sz=1024

learn.set_data(get_data(sz))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(exp_name(output_path,'unfrz',model_name,lrn,sz))

In [ ]:
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

In [ ]:
f1(preds,y)

In [ ]:
list(learn.children)[:-6]

In [ ]:
modelA = nn.Sequential(*list(learn.children)[:-7])

In [ ]:
for param in modelA.parameters():
    param.requires_grad = False

In [ ]:
inputs, labels = next(iter(dataloders['train']))
inputs, labels = Variable(inputs), Variable(labels)
outputs = res50_conv(inputs)

In [ ]:
outputs= []
def hook(module, input, output):
    outputs.append(output)

In [ ]:
learn.model[10]

In [ ]:
output = [] 
learn.model[10].register_forward_hook(hook)
out = learn.model(Variable(torch.randn(1, 3, 512, 512)).cuda())
# out = learn.model(res1)
print(outputs)

In [ ]:
torch.cuda.empty_cache()